# Set connection with GitHub

In [1]:
# ! cd /kaggle/working && rm -rf RECsys_Challenge2024

In [2]:
from kaggle_secrets import UserSecretsClient

token = UserSecretsClient().get_secret("Token")

! git clone https://{token}@github.com/madratak/RECsys_Challenge2024.git

Cloning into 'RECsys_Challenge2024'...
remote: Enumerating objects: 2438, done.
remote: Counting objects: 100% (820/820), done.
remote: Compressing objects: 100% (338/338), done.
remote: Total 2438 (delta 515), reused 732 (delta 455), pack-reused 1618 (from 1)
Receiving objects: 100% (2438/2438), 149.89 MiB | 31.15 MiB/s, done.
Resolving deltas: 100% (1460/1460), done.


In [3]:
! pip install PyGithub requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.9/375.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 28.1 MB/s eta 0:00:00


In [4]:
%cd /kaggle/working/RECsys_Challenge2024 
! python run_compile_all_cython.py

/kaggle/working/RECsys_Challenge2024
run_compile_all_cython: Found 11 Cython files in 5 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/opt/conda/bin/python'
Compiling [1/11]: MatrixFactorizationImpressions_Cython_Epoch.pyx... 
In file included from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1929,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorizationImpressions_Cython_Epoch.c:1252:
/opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
MatrixFactorizationI

In [5]:
from Utils.notebookFunctions import *
import numpy as np
import pandas as pd
import scipy.sparse as sps
import time
import shutil
import optuna
import json
import os
from Utils.seconds_to_biggest_unit import seconds_to_biggest_unit


K_PATH = '/kaggle/working/RECsys_Challenge2024'
GH_PATH = 'TrainedModels/WithoutKFCV/SingleRecommenders'

np.random.seed(42)

## Import the repository

In [6]:
repo = get_repo_from_github(token)

Repository 'RECsys_Challenge2024' found.


In [7]:
config = {
    'model': 'SLIMElasticNet',
    'tune_parameters': True,
    'database_path': '/kaggle/working/history_SLIMElasticNet.db',
    'copy_prev_best_params': False,
    'tune_best_params': True,
    'save_github': True
}

Import the database where previous tuning trials have been saved.

In [8]:
try:
    shutil.copyfile(
        f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/history_{config["model"]}.db', 
        config['database_path']
    )
except FileNotFoundError:
    pass # if not present optuna will create it

# Construction of URM and ICM matrices

In [9]:
URM_all_dataframe = pd.read_csv("/kaggle/input/recommender-system-2024-challenge-polimi/data_train.csv")

n_users = len(URM_all_dataframe["user_id"].unique())
n_items = len(URM_all_dataframe["item_id"].unique())

URM_all = sps.csr_matrix((URM_all_dataframe["data"].values, 
                          (URM_all_dataframe["user_id"].values, URM_all_dataframe["item_id"].values)),
                        shape = (n_users, n_items))

URM_all

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1764607 stored elements and shape (35736, 38121)>

In [10]:
ICM_dataframe = pd.read_csv("/kaggle/input/recommender-system-2024-challenge-polimi/data_ICM_metadata.csv")

n_items = len(ICM_dataframe["item_id"].unique())
n_features = len(ICM_dataframe["feature_id"].unique())

ICM_all = sps.csr_matrix((ICM_dataframe["data"].values, 
                          (ICM_dataframe["item_id"].values, ICM_dataframe["feature_id"].values)),
                        shape = (n_items, n_features))

ICM_all

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2940040 stored elements and shape (38121, 94331)>

# Training

In [11]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions


In [12]:
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender

def objective_function_SLIMElasticNet(optuna_trial):
    
    recommender_instance = SLIMElasticNetRecommender(URM_train)
    
    full_hyperp = {
                   "topK": optuna_trial.suggest_int("topK", 500, 1500),
                   "l1_ratio": optuna_trial.suggest_float("l1_ratio", 0.01, 1.0, log=True),
                    "alpha": optuna_trial.suggest_float("alpha", 1e-4, 1e-2, log=True),
                    "positive_only": optuna_trial.suggest_categorical("positive_only", [True, False]),
                  }        
    
    recommender_instance.fit(**full_hyperp)
    
    result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)
    
    return result_df.loc[10]["MAP"]

In [13]:
if config['tune_parameters']:
    
    optuna_study = optuna.create_study(direction='maximize', study_name=f'hyperparameters_tuning_{config["model"]}',
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)

    optuna_study.optimize(objective_function_SLIMElasticNet, n_trials=5)

[I 2024-11-25 09:15:45,111] Using an existing study with name 'hyperparameters_tuning_SLIMElasticNet' instead of creating a new one.


SLIMElasticNetRecommender: Processed 2240 ( 5.9%) in 5.00 min. Items per second: 7.46
SLIMElasticNetRecommender: Processed 4505 (11.8%) in 10.00 min. Items per second: 7.51
SLIMElasticNetRecommender: Processed 6857 (18.0%) in 15.00 min. Items per second: 7.62
SLIMElasticNetRecommender: Processed 9313 (24.4%) in 20.00 min. Items per second: 7.76
SLIMElasticNetRecommender: Processed 11657 (30.6%) in 25.01 min. Items per second: 7.77
SLIMElasticNetRecommender: Processed 14087 (37.0%) in 30.01 min. Items per second: 7.82
SLIMElasticNetRecommender: Processed 16587 (43.5%) in 35.01 min. Items per second: 7.90
SLIMElasticNetRecommender: Processed 18980 (49.8%) in 40.01 min. Items per second: 7.91
SLIMElasticNetRecommender: Processed 21465 (56.3%) in 45.01 min. Items per second: 7.95
SLIMElasticNetRecommender: Processed 23784 (62.4%) in 50.01 min. Items per second: 7.93
SLIMElasticNetRecommender: Processed 26139 (68.6%) in 55.01 min. Items per second: 7.92
SLIMElasticNetRecommender: Processed 

[I 2024-11-25 10:36:49,979] Trial 52 finished with value: 0.06009497610909174 and parameters: {'topK': 1287, 'l1_ratio': 0.1992720751977029, 'alpha': 0.00012441926440683582, 'positive_only': True}. Best is trial 49 with value: 0.06024449550387.


SLIMElasticNetRecommender: Processed 2286 ( 6.0%) in 5.00 min. Items per second: 7.62
SLIMElasticNetRecommender: Processed 4584 (12.0%) in 10.00 min. Items per second: 7.64
SLIMElasticNetRecommender: Processed 6880 (18.0%) in 15.00 min. Items per second: 7.64
SLIMElasticNetRecommender: Processed 9233 (24.2%) in 20.00 min. Items per second: 7.69
SLIMElasticNetRecommender: Processed 11509 (30.2%) in 25.00 min. Items per second: 7.67
SLIMElasticNetRecommender: Processed 13774 (36.1%) in 30.00 min. Items per second: 7.65
SLIMElasticNetRecommender: Processed 16047 (42.1%) in 35.00 min. Items per second: 7.64
SLIMElasticNetRecommender: Processed 18255 (47.9%) in 40.00 min. Items per second: 7.61
SLIMElasticNetRecommender: Processed 20606 (54.1%) in 45.01 min. Items per second: 7.63
SLIMElasticNetRecommender: Processed 22934 (60.2%) in 50.01 min. Items per second: 7.64
SLIMElasticNetRecommender: Processed 25255 (66.2%) in 55.01 min. Items per second: 7.65
SLIMElasticNetRecommender: Processed 

[I 2024-11-25 11:59:48,195] Trial 53 finished with value: 0.06025442310650729 and parameters: {'topK': 1021, 'l1_ratio': 0.15938693826337494, 'alpha': 0.000170157784618629, 'positive_only': True}. Best is trial 53 with value: 0.06025442310650729.


...

[I 2024-11-25 13:21:08,299] Trial 54 finished with value: 0.06024224130818978 and parameters: {'topK': 1193, 'l1_ratio': 0.16243668760846733, 'alpha': 0.00017181400460614633, 'positive_only': True}. Best is trial 53 with value: 0.06025442310650729.


SLIMElasticNetRecommender: Processed 2338 ( 6.1%) in 5.00 min. Items per second: 7.79
SLIMElasticNetRecommender: Processed 4710 (12.4%) in 10.00 min. Items per second: 7.85
SLIMElasticNetRecommender: Processed 7021 (18.4%) in 15.00 min. Items per second: 7.80
SLIMElasticNetRecommender: Processed 9355 (24.5%) in 20.00 min. Items per second: 7.79
SLIMElasticNetRecommender: Processed 11696 (30.7%) in 25.01 min. Items per second: 7.79
SLIMElasticNetRecommender: Processed 14136 (37.1%) in 30.01 min. Items per second: 7.85
SLIMElasticNetRecommender: Processed 16678 (43.8%) in 35.01 min. Items per second: 7.94
SLIMElasticNetRecommender: Processed 19202 (50.4%) in 40.01 min. Items per second: 8.00
SLIMElasticNetRecommender: Processed 21828 (57.3%) in 45.01 min. Items per second: 8.08
SLIMElasticNetRecommender: Processed 24398 (64.0%) in 50.01 min. Items per second: 8.13
SLIMElasticNetRecommender: Processed 27089 (71.1%) in 55.02 min. Items per second: 8.21
SLIMElasticNetRecommender: Processed 

[I 2024-11-25 14:38:14,625] Trial 55 finished with value: 0.06022574621457705 and parameters: {'topK': 1181, 'l1_ratio': 0.15502686210937466, 'alpha': 0.00018310159569481782, 'positive_only': True}. Best is trial 53 with value: 0.06025442310650729.


SLIMElasticNetRecommender: Processed 2263 ( 5.9%) in 5.00 min. Items per second: 7.54
SLIMElasticNetRecommender: Processed 4626 (12.1%) in 10.00 min. Items per second: 7.71
SLIMElasticNetRecommender: Processed 7004 (18.4%) in 15.00 min. Items per second: 7.78
SLIMElasticNetRecommender: Processed 9362 (24.6%) in 20.00 min. Items per second: 7.80
SLIMElasticNetRecommender: Processed 11715 (30.7%) in 25.01 min. Items per second: 7.81
SLIMElasticNetRecommender: Processed 14096 (37.0%) in 30.01 min. Items per second: 7.83
SLIMElasticNetRecommender: Processed 16437 (43.1%) in 35.01 min. Items per second: 7.83
SLIMElasticNetRecommender: Processed 18745 (49.2%) in 40.01 min. Items per second: 7.81
SLIMElasticNetRecommender: Processed 21139 (55.5%) in 45.01 min. Items per second: 7.83
SLIMElasticNetRecommender: Processed 23474 (61.6%) in 50.01 min. Items per second: 7.82
SLIMElasticNetRecommender: Processed 25982 (68.2%) in 55.01 min. Items per second: 7.87
SLIMElasticNetRecommender: Processed 

[I 2024-11-25 15:57:37,698] Trial 56 finished with value: 0.060260283792308925 and parameters: {'topK': 1175, 'l1_ratio': 0.1673041961676735, 'alpha': 0.0001802206495940897, 'positive_only': True}. Best is trial 56 with value: 0.060260283792308925.


## Some optuna visualizations on recommender parameters

In [14]:
if not config['tune_parameters']:
    optuna_study = optuna.load_study(study_name=f'hyperparameters_tuning_{config["model"]}', storage=f'sqlite:///{config["database_path"]}')
    
fig = optuna.visualization.plot_slice(optuna_study)
fig.show()

In [15]:
if not config['tune_parameters']:
    optuna_study = optuna.load_study(study_name=f'hyperparameters_tuning_{config["model"]}', storage=f'sqlite:///{config["database_path"]}')
    
fig = optuna.visualization.plot_param_importances(optuna_study)
fig.show()

## Let's train the recommender with best parameter values

In [16]:
if config['tune_best_params']:

    if config['tune_parameters']:
        best_params = optuna_study.best_trial.params
    else: 
        with open(f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/best_params_{config["model"]}.json', 'r') as best_params_json:
            best_params = json.load(best_params_json)

    recommender_instance = SLIMElasticNetRecommender(URM_train + URM_validation)
    recommender_instance.fit(**best_params)

SLIMElasticNetRecommender: Processed 1834 ( 4.8%) in 5.00 min. Items per second: 6.11
SLIMElasticNetRecommender: Processed 3649 ( 9.6%) in 10.00 min. Items per second: 6.08
SLIMElasticNetRecommender: Processed 5526 (14.5%) in 15.01 min. Items per second: 6.14
SLIMElasticNetRecommender: Processed 7423 (19.5%) in 20.01 min. Items per second: 6.18
SLIMElasticNetRecommender: Processed 9302 (24.4%) in 25.01 min. Items per second: 6.20
SLIMElasticNetRecommender: Processed 11158 (29.3%) in 30.01 min. Items per second: 6.20
SLIMElasticNetRecommender: Processed 13046 (34.2%) in 35.01 min. Items per second: 6.21
SLIMElasticNetRecommender: Processed 14946 (39.2%) in 40.01 min. Items per second: 6.23
SLIMElasticNetRecommender: Processed 16842 (44.2%) in 45.01 min. Items per second: 6.24
SLIMElasticNetRecommender: Processed 18664 (49.0%) in 50.01 min. Items per second: 6.22
SLIMElasticNetRecommender: Processed 20610 (54.1%) in 55.02 min. Items per second: 6.24
SLIMElasticNetRecommender: Processed 2

# Testing

Create the recommendations for the submission. 

In [17]:
if config['tune_best_params']:

    data_target_users_test = pd.read_csv('/kaggle/input/recommender-system-2024-challenge-polimi/data_target_users_test.csv')
    create_submission(data_target_users_test, recommender_instance, f'/kaggle/working/submission_{config["model"]}.csv')

Submission file saved as /kaggle/working/submission_SLIMElasticNet.csv


# Save Version on GitHub 

Write or import a json file where best hyperparameters are saved. 

In [18]:
if config['tune_parameters']:
    with open(f'/kaggle/working/best_params_{config["model"]}.json', 'w') as params_file:
        json.dump(optuna_study.best_params, params_file)
        
    if config['save_github']:
        upload_file(
            f'/kaggle/working/best_params_{config["model"]}.json', 
            f'{GH_PATH}/{config["model"]}Recommender/best_params_{config["model"]}.json', 
            f'{config["model"]} tuning results (from kaggle notebook)',
            repo
        )
elif config['copy_prev_best_params']:
    shutil.copyfile(
        f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/'\
        f'best_params_{config["model"]}.json', 
        f'/kaggle/working/best_params_{config["model"]}.json'
    )

File 'TrainedModels/WithoutKFCV/SingleRecommenders/SLIMElasticNetRecommender/best_params_SLIMElasticNet.json' updated successfully.


Save the history of the tuned model.

In [19]:
if config['save_github'] and config['tune_parameters']:
    upload_file(
        config['database_path'], 
        f'{GH_PATH}/{config["model"]}Recommender/history_{config["model"]}.db',
        f'Tuning {config["model"]} db updated results (from kaggle notebook)',
        repo
    )

File 'TrainedModels/WithoutKFCV/SingleRecommenders/SLIMElasticNetRecommender/history_SLIMElasticNet.db' updated successfully.


Save the best trained model and its submission.

In [20]:
if config['save_github'] and config['tune_best_params']: 
    upload_file(
                f'/kaggle/working/submission_{config["model"]}.csv', 
                f'{GH_PATH}/{config["model"]}Recommender/Submission/submission_{config["model"]}.csv', 
                f'New {config["model"]} submission (from kaggle notebook)',
                repo
            )

File 'TrainedModels/WithoutKFCV/SingleRecommenders/SLIMElasticNetRecommender/Submission/submission_SLIMElasticNet.csv' updated successfully.
